
$$
\text{cohesion}(n) = \left( \prod_{i=1}^{n-1} P(c_{1:i+1} | c_{1:i}) \right)^{\frac{1}{n-1}}
$$




* $cohesion(2) = P(노란 | 노)$
* $cohesion(3) = \sqrt[2]{\left( P(노란 | 노) \cdot P(노란색 | 노란) \right)}$

### Soynlp 설치

In [ ]:
!pip install soynlp

### Soynlp로 먼저 말뭉치 만들어서 학습시킨 후 명사 추출하기


In [ ]:
# 학습시킬 파일 불러오기
with open('/content/mnt/Shareddrives/Real_News/찬영/corpus.txt','r',encoding='utf-8') as file:
    text = file.readlines()

In [ ]:
# 학습시키기 (가짜뉴스와 진짜뉴스에 있는 명사를 잘 추출해내기 위함 -> mecab사전에 잘 정의되지 않는 명사들이 존재하기 때문)
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2


noun_extractor = LRNounExtractor_v2(verbose=True)
nouns = noun_extractor.train_extract(text)

In [ ]:
# 학습시킨 nouns파일로 명사를 추출해보기
tgt_noun = []
for noun in nouns:
    if (nouns[noun][0] >= 40) and (nouns[noun][1]) >= 0.5: # nouns[noun][0]은 단어의 빈도 수를 나타내고 nouns[noun][1]은 단어의 score를 나타냄 => 파라미터를 어떻게 설정해주냐에 따라 단어가 잘 추출됨을 의미
        if len(noun) == 3:   # 일단 예시로 명사길이가 세글자인 것만 추출
            tgt_noun.append(noun)

In [ ]:
tgt_noun

In [ ]:
import pandas as pd
import re

In [ ]:
# 끝에 잘못 딸린 단어들을 다시 정제해주기로 함.
non_noun_eomi_list = ('.', ',', '(',')','들','지')
tgt_noun = set([item.strip(""""',.‘“[]()”’ """) for item in tgt_noun if (item[-1] not in non_noun_eomi_list) \
                        & (sum([1 if char.isdigit() else 0 for char in item]) <= 3) \
                        & (len(re.sub('[^가-힣]', '', item)) <= 3) \
                        & (not re.search('[^가-힣0-9]', item))])

In [ ]:
# 정제된 명사 (데이터 프레임)
pd.DataFrame(tgt_noun,columns= ['noun'])

In [ ]:
# 데이터프레임으로 만든 명사를 csv파일로 저장 (경로는 사용자지정:일단 만남의 광장으로 지정)
pd.DataFrame(tgt_noun,columns= ['noun']).to_csv('/content/mnt/Shareddrives/Real_News/만남의 광장/noun.csv', encoding = 'ANSI')

### Soynlp로 만든 명사들로 Mecab 사전 추가해주기

In [ ]:
import pandas as pd

In [ ]:
# 파일 불러오기
mecab_dict = pd.read_csv('/content/mnt/Shareddrives/Real_News/찬영/noun.csv',encoding='cp949')

In [ ]:
# 먼저 mecab 설치하기
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

In [ ]:
! bash install_mecab-ko_on_colab190912.sh

In [ ]:
# mecab 불러오기
from konlpy.tag import Mecab

사전 추가하기 전 사전작업 (설치)

In [ ]:
cd /content/mecab-ko-dic-2.1.1-20180720

/content/mecab-ko-dic-2.1.1-20180720


In [ ]:
!pip install jamo

In [ ]:
# 종성 여부 함수 정의
from jamo import h2j, j2hcj 

def get_jongsung_TF(sample_text):
   sample_text_list = list(sample_text) 
   last_word = sample_text_list[-1] 
   last_word_jamo_list = list(j2hcj(h2j(last_word))) 
   last_jamo = last_word_jamo_list[-1] 
   jongsung_TF = "T" 

   if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
     jongsung_TF = "F" 
   return jongsung_TF


In [ ]:
# mecab_dict 정의 

with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f:  # 여러 파일들이 있는데 저는 이 파일에 넣어줌.
  file_data = f.readlines()

word_list = mecab_dict['0'].to_list()  # column명이 0으로 되어있어서 0으로 넣어줌

# line 부분에 0,1,2는 우선순위를 넣어주기 위함(숫자가 낮아질수록 가장먼저 토큰화 됨.)
for word in word_list:
  jongsung_TF = get_jongsung_TF(word)
  if len(word)>=4:
    line = '{},,,0,NNP,*,{},{},*,*,*,*,*\n'.format(word , jongsung_TF, word)
    file_data.append(line)
    
  elif len(word) == 3:
    line = '{},,,1,NNP,*,{},{},*,*,*,*,*\n'.format(word , jongsung_TF, word)
    file_data.append(line)

  elif len(word) == 2:
    line = '{},,,2,NNP,*,{},{},*,*,*,*,*\n'.format(word , jongsung_TF, word)
    file_data.append(line)


In [ ]:
# 잘 정의되었는지 보기

with open("./user-dic/nnp.csv", 'w', encoding='utf-8') as f:
  for line in file_data:
    f.write(line)

##### 사전 추가를 한 후 다시 재업로드 해줘야 됨. => 그래야 사전 추가된 인식 가능

In [ ]:
ls tools

add-userdic.sh*  convert_for_using_store.sh*  mecab-bestn.sh*


In [ ]:
!bash ./tools/add-userdic.sh

In [ ]:
user_nnp = pd.read_csv('/content/mecab-ko-dic-2.1.1-20180720/user-nnp.csv')
user_nnp

In [ ]:
for i in user_nnp.대우:
  if len(i)>=4:
    user_nnp.loc[(user_nnp.대우 == i),'3821'] = 0

  elif len(i) == 3:
    user_nnp.loc[(user_nnp.대우 == i),'3821'] = 1

  elif len(i) == 2:
    user_nnp.loc[(user_nnp.대우 == i),'3821'] = 2

In [ ]:
user_nnp.to_csv('/content/mecab-ko-dic-2.1.1-20180720/user-nnp.csv',index=False)

In [ ]:
!make clean

In [ ]:
!make install

In [ ]:
# 이제 mecab함수를 불러와서 토큰화 진행하면 됨.
mecab=Mecab()

In [ ]:
print(mecab.pos('내일 폭등 예상되는 극비 종목 BEST 5 .  눌러서 확인하기'))

[('내일', 'NNP'), ('폭등', 'NNP'), ('예상', 'NNP'), ('되', 'VV'), ('는', 'ETM'), ('극비', 'NNG'), ('종목', 'NNP'), ('BEST', 'SL'), ('5', 'SN'), ('.', 'SF'), ('눌러서', 'VV+EC'), ('확인', 'NNG'), ('하', 'XSV'), ('기', 'ETN')]


In [ ]:
print(mecab.pos('내일 폭등 기대주  1등 종목 무료 확인하기'))

[('내일', 'NNP'), ('폭등', 'NNP'), ('기대', 'NNP'), ('주', 'NNP'), ('1등', 'NNP'), ('종목', 'NNP'), ('무료', 'NNP'), ('확인', 'NNG'), ('하', 'XSV'), ('기', 'ETN')]
